In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt#
import evaluate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer
from tensorflow.keras.optimizers import Adam
from datasets import load_dataset
from transformers import create_optimizer,T5TokenizerFast,DataCollatorForSeq2Seq,TFT5ForConditionalGeneration,TFAutoModelForSeq2SeqLM,AutoModelForSeq2SeqLM,TFT5ForConditionalGeneration

2024-08-01 19:09:28.363523: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 19:09:28.363596: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 19:09:28.364777: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-01 19:09:28.371813: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-01 19:09:29.258223: W tensorflow/compiler/tf2

In [2]:
BATCH_SIZE=32
MAX_LENGTH=128

## Data Preparation

In [3]:
dataset_id="leslyarun/c4_200m_gec_train100k_test25k"

In [4]:
dataset = load_dataset(dataset_id)

In [5]:
dataset['train'][0]

{'input': 'Bitcoin is for $7,094 this morning, which CoinDesk says.',
 'output': 'Bitcoin goes for $7,094 this morning, according to CoinDesk.'}

In [6]:
model_id="t5-small"
tokenizer=T5TokenizerFast.from_pretrained(model_id)

In [7]:
def preprocess_function(examples):

  inputs = [example for example in examples['input']]
  targets = [example for example in examples['output']]

  model_inputs = tokenizer(inputs, text_target=targets,max_length=MAX_LENGTH, truncation=True)
  return model_inputs

In [8]:
tokenized_dataset=dataset.map(preprocess_function,batched=True,remove_columns=dataset["train"].column_names)

In [9]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_id)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,model=model, return_tensors="tf")

2024-08-01 19:09:39.010795: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-01 19:09:39.011854: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-01 19:09:39.011919: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-01 19:09:39.015007: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-01 19:09:39.015075: I external/local_xla/xla/stream_executor

In [10]:
train_dataset=tokenized_dataset["train"].to_tf_dataset(
    shuffle=True,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
)

2024-08-01 19:09:40.414896: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [11]:
val_dataset=tokenized_dataset["test"].to_tf_dataset(
    shuffle=False,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
)

## Modeling

In [12]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60506624 (230.81 MB)
Trainable params: 60506624 (230.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
num_epochs = 1
num_train_steps=len(train_dataset)*num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
)
model.compile(optimizer=optimizer)

In [14]:
history=model.fit(
  train_dataset,
  validation_data=val_dataset,
  epochs=1
)

2024-08-01 19:10:15.647343: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.96GiB (rounded to 2105540608)requested by op gradient_tape/tft5_for_conditional_generation/MatMul/MatMul_1
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-08-01 19:10:15.647428: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-08-01 19:10:15.647448: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 111, Chunks in use: 109. 27.8KiB allocated for chunks. 27.2KiB in use in bin. 729B client-requested in use in bin.
2024-08-01 19:10:15.647456: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 1, Chunks in use: 0. 768B allocated for chunks. 0B in use in bin. 0B client-requeste

ResourceExhaustedError: Graph execution error:

Detected at node gradient_tape/tft5_for_conditional_generation/MatMul/MatMul_1 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/dhruv/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/dhruv/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/dhruv/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/dhruv/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/dhruv/.local/lib/python3.10/site-pa

## Testing on pretrained model

In [15]:
pretrained_model=AutoModelForSeq2SeqLM.from_pretrained(
    "juancavallotti/t5-base-gec"
)

config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [16]:
wrong_english=[
    "Dady hav'e eateing her foot",
    "DJ Sorryyouwastedyourmoneytobehere",
    "i used to like to swimming",
    "maybe we should organized a meetin with the people from unesco",
    "when are we goinge to start play football",
    "many a time rain fall in my city",
   ]
tokenized=tokenizer(
  wrong_english,
  padding="longest",
  max_length=MAX_LENGTH,
  truncation=True,
  return_tensors='pt'
)
out=pretrained_model.generate(**tokenized, max_length=128)
print(out)

tensor([[    0, 12919,    63,     3,   342,   160,  2418,     3,     5,     1,
             0,     0,     0,     0,     0,     0,     0],
        [    0,  6920, 11342,     6,    25, 21133,    39,   540,     3,     6,
             1,     0,     0,     0,     0,     0,     0],
        [    0,    27,   261,    12,   114,  5989,     3,     5,     1,     0,
             0,     0,     0,     0,     0,     0,     0],
        [    0,  3836,    62,   225,  7958,     3,     9,  1338,    28,     8,
           151,    45,   245,  3523,     3,     5,     1],
        [    0,   366,    33,    62,   352,    12,   456,  1556,  3370,     1,
             0,     0,     0,     0,     0,     0,     0],
        [    0,  1404,     3,     9,    97,  3412,  7250,    16,    82,   690,
             1,     0,     0,     0,     0,     0,     0]])


In [17]:
for i in range(len(wrong_english)):
  print(wrong_english[i]+"------------>"+tokenizer.decode(out[i], skip_special_tokens=True))

Dady hav'e eateing her foot------------>Dady ate her foot.
DJ Sorryyouwastedyourmoneytobehere------------>DJ Sorry, you wasted your money,
i used to like to swimming------------>I used to like swimming.
maybe we should organized a meetin with the people from unesco------------>Maybe we should organize a meeting with the people from unesco.
when are we goinge to start play football------------>When are we going to start playing football
many a time rain fall in my city------------>Many a time rain falls in my city
